<a href="https://colab.research.google.com/github/giirrr/first_ryun_project/blob/main/AE_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mlp
import warnings
import sklearn
import os
import tensorflow as tf
import tensorflow.keras as keras
import random
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import LSTM, Dropout, Dense, Activation, Conv1D, Flatten, Lambda, Reshape, concatenate, Input
import datetime
import seaborn as sb
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dropout,Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM, GRU, SimpleRNN
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from keras import callbacks

"""##전처리"""

random.sample(range(120), 24)

df_train = pd.read_csv('./data/master_data.csv')     # 7~8월
df_test = pd.read_csv('./data/test_data.csv')        # 9월1일

df_train[df_train['AIR_PRESSURE'] < 900] = np.nan
df_train[df_train['AIR_PRESSURE'] > 1100 ] = np.nan

df_train[df_train['AIR_TEMPERATURE'] < -50] = np.nan
df_train[df_train['AIR_TEMPERATURE'] > 70 ] = np.nan

df_train[df_train['HUMIDITY'] < 0] = np.nan
df_train[df_train['HUMIDITY'] > 100 ] = np.nan

df_train[df_train['WIND_SPEED'] < 0] = np.nan
df_train[df_train['WIND_SPEED'] > 80 ] = np.nan

df_test[df_test['AIR_PRESSURE'] < 900] = np.nan
df_test[df_test['AIR_PRESSURE'] > 1100 ] = np.nan

df_test[df_test['AIR_TEMPERATURE'] < -50] = np.nan
df_test[df_test['AIR_TEMPERATURE'] > 70 ] = np.nan

df_test[df_test['HUMIDITY'] < 0] = np.nan
df_test[df_test['HUMIDITY'] > 100 ] = np.nan

df_test[df_test['WIND_SPEED'] < 0] = np.nan
df_test[df_test['WIND_SPEED'] > 80 ] = np.nan

"""변동값 테스트"""
# df_train['AIR_TEMPERATURE_delta'] = df_train['AIR_TEMPERATURE']-df_train['AIR_TEMPERATURE'].shift(1)
# df_train['AIR_PRESSURE_delta'] = df_train['AIR_PRESSURE']-df_train['AIR_PRESSURE'].shift(1)
# df_train['HUMIDITY_delta'] = df_train['HUMIDITY']-df_train['HUMIDITY'].shift(1)
# df_train['WIND_SPEED_delta'] = df_train['WIND_SPEED']-df_train['WIND_SPEED'].shift(1)
#
# df_train[df_train['AIR_TEMPERATURE_delta']>(df_train['AIR_TEMPERATURE_delta'].mean()*2)] = np.nan
# df_train[df_train['AIR_PRESSURE_delta']>(df_train['AIR_PRESSURE_delta'].mean()*2)] = np.nan
# df_train[df_train['HUMIDITY_delta']>(df_train['HUMIDITY_delta'].mean()*2)] = np.nan
# df_train[df_train['WIND_SPEED_delta']>(df_train['WIND_SPEED_delta'].mean()*2)] = np.nan
#
df_train = df_train.dropna()
df_test = df_test.dropna()
#df_train.info()
#df_test.info()


#MinMaxScaler 전처리
df_train['AIR_PRESSURE'] = (lambda ap : (ap-900) / 200)(df_train['AIR_PRESSURE'])
df_test['AIR_PRESSURE'] = (lambda ap : (ap-900) / 200)(df_test['AIR_PRESSURE'])
df_train['AIR_TEMPERATURE'] = (lambda at : (at+50) / 120)(df_train['AIR_TEMPERATURE'])
df_test['AIR_TEMPERATURE'] = (lambda at : (at+50) / 120)(df_test['AIR_TEMPERATURE'])
df_train['day_min'] = (lambda dm : dm / 1439)(df_train['day_min'])
df_test['day_min'] = (lambda dm : dm / 1439)(df_test['day_min'])
df_train['HUMIDITY'] = (lambda h : h / 100)(df_train['HUMIDITY'])
df_test['HUMIDITY'] = (lambda h : h / 100)(df_test['HUMIDITY'])
df_train['WIND_SPEED'] = (lambda ws : ws / 80)(df_train['WIND_SPEED'])
df_test['WIND_SPEED'] = (lambda ws : ws / 80)(df_test['WIND_SPEED'])
#StandardScaler 전처리


feature_cols = ['AIR_TEMPERATURE','AIR_PRESSURE', 'day_min', 'HUMIDITY', 'WIND_SPEED']
label_cols = ['AIR_TEMPERATURE']

y_train_df = df_train[label_cols].values
X_train_df = df_train[feature_cols].values
y_test_df = df_test[label_cols].values
X_test_df = df_test[feature_cols].values

xr_length = len(X_train_df[:-24])
X_train = np.zeros((xr_length, 120))
yr_length = len(y_train_df[:-24])
y_train = np.zeros((yr_length, 25))

for rl in range(xr_length):
  X_train[rl] = X_train_df[rl:rl+24].reshape((120))
  y_train[rl] = y_train_df[rl:rl+25].reshape((25))
  for rs in random.sample(range(120), 24):
    X_train[rl, rs] = 0

xr_length = len(X_test_df[:-24])
X_test = np.zeros((xr_length, 120))
yr_length = len(y_test_df[:-24])
y_test = np.zeros((yr_length, 25))

for rl in range(xr_length):
  X_test[rl] = X_test_df[rl:rl+24].reshape((120))
  y_test[rl] = y_test_df[rl:rl+25].reshape((25))
  for rs in random.sample(range(120), 24):
    X_test[rl, rs] = 0


"""##Model 쌓기"""

input_size = 120

input = Input(shape=(input_size))
x = Dense(64, activation='sigmoid')(input)
x = Dense(32, activation='sigmoid')(x)
x = Reshape((32, 1))(x)
x = LSTM(16, activation='sigmoid')(x)
x = Dense(32, activation='sigmoid')(x)
x = Dense(64, activation='sigmoid')(x)
out = Dense(25, activation='sigmoid')(x)
model = Model(input, [out], name='CGNN_cycle')

model.compile(loss='msle',optimizer='Nadam')

print(model.summary())

if 'AE-LSTM' in os.listdir():
    pass
else:
    os.mkdir('AE-LSTM')

ep = 1000
callbackz = [callbacks.ModelCheckpoint("./AE-LSTM/{loss:.8f}-{val_loss:.8f}-{epoch:04d}.h5",
                                       monitor="val_loss",
                                       save_best_only=False,
                                       save_freq='epoch'
                                       )]
history = model.fit(X_train, y_train,
                    batch_size = 16,
                    validation_split=0.1,
                    epochs=ep,
                    callbacks=callbackz)

# 7 훈련 과정 시각화 (손실)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('AE-LSTM Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

"""##정확도"""

train_acc = model.evaluate(X_train,y_train)
test_acc = model.evaluate(X_test,y_test)

print(train_acc)
print(test_acc)

print(y_test)
print(y_test.shape)
y_pred = model.predict(X_test)
print(y_pred)
print(y_pred.shape)

"""##점예측 시각화"""

y_test2 =(lambda at : (at*120)-50)(y_test)

y_pred2 =(lambda at : (at*120)-50)(y_pred)

fig = plt.figure(figsize=(20, 10), facecolor ='white')
x = np.arange(len(y_pred))
plt.title('AutoEncoder-LSTM')
plt.plot(x, y_test2[:,-1], label= 'actual')
plt.plot(x, y_pred2[:,-1], label= 'Prediction')
plt.legend()
plt.show()


